In [1]:
!pip install pendulum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 17.4 MB/s eta 0:00:00


In [ ]:
#Import Library
import numpy as np
import pandas as pd
from datetime import datetime
import calendar 
import pendulum
from openpyxl import load_workbook

#Import Data
data_raw = pd.read_excel('2023-04-22_Data Inventory FL Indonesia - Wk 12-2023.xlsx', sheet_name='Sheet1')
data_sku_product = pd.read_excel('SKU.xlsx', sheet_name='Product')
data_sku_sales = pd.read_excel('SKU.xlsx', sheet_name='Sales')
data_sku_inventory = pd.read_excel('SKU.xlsx', sheet_name='Inventory')
data_sku_mapping = pd.read_excel('SKU.xlsx', sheet_name='Mapping')

#Create Temporary List
gen_norm = []
stock_norm = []
date = []
week = []
store_name = []
division = []
subdiv_code = []
subdiv = []
gender = []
vendor_art = []
month_month = []
brand_name = []
franchise = []
store_type = []
color = []
stock_retail = []
stock_cost = []

#Missing Variables
missing_store_site = []
missing_article_product = []

#Filter Sales Quantity
data_raw = data_raw.drop(data_raw[data_raw['StockQty'].isin([0])].index)  
data_raw = data_raw.reset_index(drop=True)

#Drop Duplicates But Keep Highest Value for StockCost
data_stock_cost = data_raw[['Genarticle', 'StockQty', 'StockCost']].copy(deep=True)
data_stock_cost['Cost'] = np.where(data_stock_cost['StockQty'] == 0, data_stock_cost['StockCost'], data_stock_cost['StockCost']/data_stock_cost['StockQty'])
data_stock_cost = data_stock_cost.groupby('Genarticle', group_keys=False).apply(lambda x: x.loc[x.Cost.idxmax()])
data_stock_cost = data_stock_cost.reset_index(drop=True)

#Drop Duplicates But Keep Highest Value for StockRetail
data_stock_retail = data_raw[['Genarticle', 'StockQty', 'CurrentPrice']].copy(deep=True)
data_stock_retail = data_stock_retail.groupby('Genarticle', group_keys=False).apply(lambda x: x.loc[x.CurrentPrice.idxmax()])
data_stock_retail = data_stock_retail.reset_index(drop=True)

for i in range(len(data_raw)) : 
  #Delimit Date
  date.append(datetime.strptime(data_raw['InventoryDate'][i], '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y'))

  #Month Month
  num_month = date[i].split('/')[1]
  month_form = str(num_month).zfill(2) + " - " + calendar.month_abbr[int(num_month)]
  month_month.append(month_form)

  #Week
  num_week = int(datetime.strptime(date[i], '%d/%m/%Y').date().strftime("%U"))
  if 49 <= num_week >= 52 : 
    num_week += 48
  else : 
    num_week -= 4
  num_week = str(num_week).zfill(2)
  week_name = "Wk " + num_week
  week.append(week_name)

  #Normalize GenArt and Stock Norm
  gen_art = str(data_raw['Genarticle'][i])
  if gen_art.find('MB-') == -1 : 
    gen_norm.append(gen_art)
    stock_norm.append(int(data_raw['StockQty'][i]))
  else : 
    gen_art = gen_art[0:3] + gen_art[6:(6+(len(gen_art)-9))]
    gen_norm.append(gen_art)
    
    brand_code = str(data_raw['BrandCode'][i])
    if brand_code == 'JDK' or brand_code == 'NYA' or brand_code == 'CVK' :
      lookup_value = str(data_raw['Genarticle'][i])[-3:]
      stock_loc = data_sku_mapping.index[data_sku_mapping['Code 2']==lookup_value].tolist()
      if len(stock_loc)>0 :
        stock_norm.append(int(data_sku_mapping['qty'][stock_loc[0]])*int(data_raw['StockQty'][i]))
      else :
        missing_article_product.append(gen_art)
        stock_norm.append("#N/A")
    else :
      lookup_value = brand_code + str(data_raw['Genarticle'][i])[-3:]
      stock_loc = data_sku_mapping.index[data_sku_mapping['Code 1']==lookup_value].tolist()
      if len(stock_loc)>0 :
        stock_norm.append(int(data_sku_mapping['qty'][stock_loc[0]])*int(data_raw['StockQty'][i]))
      else :
        missing_article_product.append(gen_art)
        stock_norm.append("#N/A")

  #Store Name and Type
  site_loc = data_sku_sales.index[data_sku_sales['Site Code 1']==data_raw['Site'][i]].tolist()
  if len(site_loc)>0 :
    store_name.append(data_sku_sales['Desc 1'][site_loc[0]])
    store_type.append(data_sku_sales['Store Type'][site_loc[0]])
  else :
    site_loc = data_sku_inventory.index[data_sku_inventory['Site Code 2']==data_raw['Site'][i]].tolist()
    if len(site_loc)>0 :
      store_name.append(data_sku_inventory['Desc 2'][site_loc[0]])
      store_type.append(data_sku_inventory['Notes 3'][site_loc[0]])
    else : 
      missing_store_site.append(data_raw['Site'][i])
      store_name.append("#N/A")
      store_type.append("#N/A")
  
  #Division to Color
  #FL Indonesia
  genarticle_loc = data_sku_product.index[data_sku_product['Gen.art']==gen_norm[i]].tolist()

  #FL Philippines
  # genarticle_loc = data_sku_product.index[data_sku_product['Gen.art (PH)']==gen_norm[i]].tolist()

  if len(genarticle_loc)>0 :
    division.append(data_sku_product['FL Division'][genarticle_loc[0]])
    subdiv_code.append(data_sku_product['FL Subdivision Code'][genarticle_loc[0]])
    subdiv.append(data_sku_product['FL Subdivision'][genarticle_loc[0]])
    gender.append(data_sku_product['FL Gender / Age'][genarticle_loc[0]])
    vendor_art.append(data_sku_product['Vendor Art'][genarticle_loc[0]])
    brand_name.append(data_sku_product['Brand Name'][genarticle_loc[0]])
    franchise.append(data_sku_product['Franchise/Silhoutte'][genarticle_loc[0]])
    color.append(data_sku_product['Color Desc'][genarticle_loc[0]])
  else :
    missing_article_product.append(gen_norm[i])
    division.append("#N/A")
    subdiv_code.append("#N/A")
    subdiv.append("#N/A")
    gender.append("#N/A")
    vendor_art.append("#N/A")
    brand_name.append("#N/A")
    franchise.append("#N/A")
    color.append("#N/A")
  
  #StockCost
  if len(str(int(float(data_raw['StockCost'][i])))) <= 2 : 
    cost_loc = data_stock_cost.index[data_stock_cost['Genarticle']==data_raw['Genarticle'][i]].tolist()
    if len(cost_loc) > 0 : 
      if data_stock_cost['Cost'][cost_loc[0]] == 0 : 
        cost_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
        if len(cost_loc) > 0 : 
          stock_cost.append(int(float(data_raw['StockQty'][i] * data_sku_product[' Est Landed Cost '][cost_loc[0]])))
        else : 
          stock_cost.append("#N/A")
      else :   
        stock_cost.append(int(float(data_raw['StockQty'][i] * data_stock_cost['Cost'][cost_loc[0]])))
    else : 
      stock_cost.append("#N/A")
  else : 
    stock_cost.append(int(float(data_raw['StockCost'][i])))

  #Stock Retail
  current_price = int(float(data_raw['CurrentPrice'][i]))
  if stock_norm[i] != "#N/A" :
    if current_price != 0 :
      temp_retail_price = int(data_raw['StockQty'][i]) * current_price
      if temp_retail_price - int(float(data_raw['StockRetail'][i])) == 0 : 
        stock_retail.append(int(float(data_raw['StockRetail'][i])))
      else : 
        stock_retail.append(temp_retail_price)
    else : 
      price_loc = data_stock_retail.index[data_stock_retail['Genarticle']==data_raw['Genarticle'][i]].tolist()
      if len(price_loc)>0 :
        if data_stock_retail['CurrentPrice'][price_loc[0]] != 0 : 
          temp_retail_price = int(data_raw['StockQty'][i]) * data_stock_retail['CurrentPrice'][price_loc[0]]
          stock_retail.append(temp_retail_price)
        else :
          if int(float(data_raw['StockRetail'][i])) != 0 : 
            stock_retail.append(int(float(data_raw['StockRetail'][i])))
          else :
            price_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
            if len(price_loc) > 0 : 
              temp_retail_price = int(data_raw['StockQty'][i]) * int(float(data_sku_product[' Retail Price '][price_loc[0]]))
              stock_retail.append(temp_retail_price)
            else : 
              stock_retail.append("#N/A")
      else : 
        price_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
        if len(price_loc) > 0 : 
          temp_retail_price = int(data_raw['StockQty'][i]) * int(float(data_sku_product[' Retail Price '][price_loc[0]]))
          stock_retail.append(temp_retail_price)
        else : 
          stock_retail.append("#N/A")
  else : 
    stock_retail.append("#N/A")

#Combine All
data_final = data_raw.copy(deep=True)
data_final['StockCost'] = stock_cost
data_final['Gen Norm'] = gen_norm
data_final['Stock Norm'] = stock_norm
data_final['Date'] = date
data_final['Week'] = week
data_final['Store Name'] = store_name
data_final['FL Division'] = division
data_final['FL SubDiv Code'] = subdiv_code
data_final['FL SubDiv'] = subdiv
data_final['FL Gender / Age'] = gender
data_final['Vendor Art'] = vendor_art
data_final['Month Desc'] = month_month
data_final['BRAND'] = brand_name
data_final['Franchise (FTW)'] = franchise
data_final['Color'] = color
data_final['Store Type'] = store_type
data_final['Stock Retail 1'] = stock_retail

#Export Data
data_final.to_excel("Hasil 2023-04-22_Data Inventory FL Indonesia - Wk 12-2023.xlsx", sheet_name = 'FINAL')

if len(missing_store_site) > 0:
  print("Terdapat Missing Store Site : %d Baris" % (len(missing_store_site)))
  missing_store_site = pd.DataFrame(missing_store_site, columns =['Store Site'])
  missing_store_site.to_excel('Missing Store Site.xlsx', sheet_name='Store Site Missing')
else : 
  print("Tidak Terdapat Missing Store Site : %d Baris" % (len(missing_store_site)))

if len(missing_article_product) > 0:
  print("Terdapat Missing Article Product : %d Baris" % (len(missing_article_product)))
  missing_article_product = pd.DataFrame(missing_article_product, columns =['Article Product'])
  missing_article_product.to_excel('Missing Article Product.xlsx', sheet_name='Article Product Missing')
else : 
  print("Tidak Missing Article Product : %d Baris" % (len(missing_article_product)))